In [ ]:
!pip install requests-aws4auth

In [1]:
import boto3
import requests
import json
from requests_aws4auth import AWS4Auth

aoss_client = boto3.client('opensearchserverless')

In [ ]:
coll_list_response = aoss_client.list_collections()

matching_collections = [
    collection for collection in coll_list_response.get("collectionSummaries", [])
    if collection['name'] == 'genai-vectors-collection'
]

assert len(matching_collections) == 1, "Expected exactly 1 collection with specified name"

collection = matching_collections[0]
collection_name = collection["name"]
collection_id = collection["id"]  # Get the collection ID

# Now, fetch the collection details using batch_get_collection
collection_details_response = aoss_client.batch_get_collection(ids=[collection_id])

# Extract collection endpoint
collection_detail = collection_details_response.get("collectionDetails", [])[0]
collection_endpoint = collection_detail.get("collectionEndpoint")

print(f"Found Collection: {collection['name']}")
print(f"Endpoint: {collection_endpoint}")

In [ ]:
INDEX_NAME = "genai-vectors-collection"

index_mapping = {
  "settings": {
    "index.knn": True
  },
  "mappings": {
    "properties": {
      "vector-field": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "engine": "faiss",
          "name": "hnsw"
        }
      },
      "text": {
        "type": "text"
      },
      "metadata": {
        "type": "keyword"
      }
    }
  }
}

# OpenSearch API URL for index creation
index_url = f"{collection_endpoint}/{INDEX_NAME}"

# OpenSearch requires authentication (IAM or Basic Auth)
headers = {
    "Content-Type": "application/json"
}

session = boto3.Session()
credentials = session.get_credentials()
aws_auth = AWS4Auth(
    credentials.access_key, credentials.secret_key,
    session.region_name, "aoss",
    session_token=credentials.token
)

# Send request to create index
response = requests.put(index_url, auth=aws_auth, headers=headers, data=json.dumps(index_mapping))

# Print response
print("Index Creation Response:", response.status_code, response.text)